In [ ]:
import pandas as pd

In [2]:
df_total = pd.read_csv('../data/raw/2-ExploracionDataframes.csv', sep=';')
df_total

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,Provincia Destino,CUT Region Destino,Region Destino,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino
0,1101,Iquique,11,Iquique,1,Tarapacá,1402,Camiña,Camiña,14,Tamarugal,1,Tarapacá,2019,1,enero,109,Alta,1061.580205,1061.580205
1,1101,Iquique,11,Iquique,1,Tarapacá,1403,Colchane,Colchane - P.N. Volcán Isluga,14,Tamarugal,1,Tarapacá,2019,1,enero,298,Alta,1061.580205,1061.580205
2,1101,Iquique,11,Iquique,1,Tarapacá,1404,Huara,Resto región Tarapacá,14,Tamarugal,1,Tarapacá,2019,1,enero,2594,Alta,1061.580205,1061.580205
3,1101,Iquique,11,Iquique,1,Tarapacá,1405,Pica,Pica,14,Tamarugal,1,Tarapacá,2019,1,enero,4138,Alta,1061.580205,1061.580205
4,1101,Iquique,11,Iquique,1,Tarapacá,2101,Antofagasta,Antofagasta,21,Antofagasta,2,Antofagasta,2019,1,enero,2071,Alta,1061.580205,3724.530431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137551,16305,San Nicolás,163,Punilla,16,Ñuble,16202,Cobquecura,Cobquecura,162,Itata,16,Ñuble,2024,12,diciembre,36,Alta,NaN,NaN
3137552,16305,San Nicolás,163,Punilla,16,Ñuble,16203,Coelemu,Valle del Itata,162,Itata,16,Ñuble,2024,12,diciembre,16,Alta,NaN,NaN
3137553,16305,San Nicolás,163,Punilla,16,Ñuble,16207,Treguaco,Resto región Ñuble,162,Itata,16,Ñuble,2024,12,diciembre,52,Alta,NaN,NaN
3137554,16305,San Nicolás,163,Punilla,16,Ñuble,16302,Coihueco,Resto región Ñuble,163,Punilla,16,Ñuble,2024,12,diciembre,85,Alta,NaN,NaN


In [3]:
# Agrupar los viajes por 'Region Destino' y 'Mes'
viajes_totales_temporada = df_total.groupby(['Region Destino', 'Provincia Destino','Comuna Destino', 'Anio','CUT Mes','Mes', 'Temporada'])['Viajes Ocasionales'].sum().reset_index()

# Lista de niveles de agregación para los totales
niveles_agrupacion = [
    (['Region Destino', 'Anio', 'Temporada'], 'Total Temporada Region'),
    (['Provincia Destino', 'Anio', 'Temporada'], 'Total Temporada Provincia'),
    (['Comuna Destino', 'Anio', 'Temporada'], 'Total Temporada Comuna')
]

# Iterar sobre los niveles de agrupación y añadir las columnas de totales
for nivel, nombre_columna in niveles_agrupacion:
    # Crear un dataframe con el total de viajes por el nivel de agregación
    total_viajes = viajes_totales_temporada.groupby(nivel)['Viajes Ocasionales'].sum().reset_index()
    total_viajes.rename(columns={'Viajes Ocasionales': nombre_columna}, inplace=True)
    
    # Unir este total al dataframe original
    viajes_totales_temporada = viajes_totales_temporada.merge(total_viajes, on=nivel, how='left')

viajes_totales_temporada.head()

,Region Destino,Provincia Destino,Comuna Destino,Anio,CUT Mes,Mes,Temporada,Viajes Ocasionales,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
0,Antofagasta,Antofagasta,Antofagasta,2019,1,enero,Alta,46060,500988,301193,187990
1,Antofagasta,Antofagasta,Antofagasta,2019,2,febrero,Alta,36583,500988,301193,187990
2,Antofagasta,Antofagasta,Antofagasta,2019,3,marzo,Baja,15803,327962,190831,130066
3,Antofagasta,Antofagasta,Antofagasta,2019,4,abril,Baja,17685,327962,190831,130066
4,Antofagasta,Antofagasta,Antofagasta,2019,5,mayo,Baja,31024,327962,190831,130066


In [4]:
# Eliminar la columna 'Viajes Ocasionales' del dataframe 'viajes_por_mes_region' antes del merge
viajes_totales_temporada = viajes_totales_temporada.drop(columns=['Viajes Ocasionales'])


# Realizar el merge del dataframe 'viajes_por_mes_region' al dataframe original 'df'
df_total = df_total.merge(
    viajes_totales_temporada, 
    on=['Region Destino', 'Provincia Destino', 'Comuna Destino', 'Anio', 'CUT Mes', 'Mes', 'Temporada'], 
    how='left'
)

df_total

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,...,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
0,1101,Iquique,11,Iquique,1,Tarapacá,1402,Camiña,Camiña,14,...,2019,1,enero,109,Alta,1061.580205,1061.580205,268129,76446,1276
1,1101,Iquique,11,Iquique,1,Tarapacá,1403,Colchane,Colchane - P.N. Volcán Isluga,14,...,2019,1,enero,298,Alta,1061.580205,1061.580205,268129,76446,5512
2,1101,Iquique,11,Iquique,1,Tarapacá,1404,Huara,Resto región Tarapacá,14,...,2019,1,enero,2594,Alta,1061.580205,1061.580205,268129,76446,22586
3,1101,Iquique,11,Iquique,1,Tarapacá,1405,Pica,Pica,14,...,2019,1,enero,4138,Alta,1061.580205,1061.580205,268129,76446,21894
4,1101,Iquique,11,Iquique,1,Tarapacá,2101,Antofagasta,Antofagasta,21,...,2019,1,enero,2071,Alta,1061.580205,3724.530431,500988,301193,187990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137551,16305,San Nicolás,163,Punilla,16,Ñuble,16202,Cobquecura,Cobquecura,162,...,2024,12,diciembre,36,Alta,NaN,NaN,771617,135805,36076
3137552,16305,San Nicolás,163,Punilla,16,Ñuble,16203,Coelemu,Valle del Itata,162,...,2024,12,diciembre,16,Alta,NaN,NaN,771617,135805,19654
3137553,16305,San Nicolás,163,Punilla,16,Ñuble,16207,Treguaco,Resto región Ñuble,162,...,2024,12,diciembre,52,Alta,NaN,NaN,771617,135805,9059
3137554,16305,San Nicolás,163,Punilla,16,Ñuble,16302,Coihueco,Resto región Ñuble,163,...,2024,12,diciembre,85,Alta,NaN,NaN,771617,143109,19588


In [5]:
# Filtrar el dataframe para eliminar filas que cumplen con las condiciones
df_filtrado = df_total[~((df_total['Anio'] == 2024) & (df_total['CUT Mes'] > 6))]
df_total = df_filtrado
df_total

,CUT Comuna Origen,Comuna Origen,CUT Provincia Origen,Provincia Origen,CUT Region Origen,Region Origen,CUT Comuna Destino,Comuna Destino,Destino Turistico,CUT Provincia Destino,...,Anio,CUT Mes,Mes,Viajes Ocasionales,Temporada,PIB Region Origen,PIB Region Destino,Total Temporada Region,Total Temporada Provincia,Total Temporada Comuna
0,1101,Iquique,11,Iquique,1,Tarapacá,1402,Camiña,Camiña,14,...,2019,1,enero,109,Alta,1061.580205,1061.580205,268129,76446,1276
1,1101,Iquique,11,Iquique,1,Tarapacá,1403,Colchane,Colchane - P.N. Volcán Isluga,14,...,2019,1,enero,298,Alta,1061.580205,1061.580205,268129,76446,5512
2,1101,Iquique,11,Iquique,1,Tarapacá,1404,Huara,Resto región Tarapacá,14,...,2019,1,enero,2594,Alta,1061.580205,1061.580205,268129,76446,22586
3,1101,Iquique,11,Iquique,1,Tarapacá,1405,Pica,Pica,14,...,2019,1,enero,4138,Alta,1061.580205,1061.580205,268129,76446,21894
4,1101,Iquique,11,Iquique,1,Tarapacá,2101,Antofagasta,Antofagasta,21,...,2019,1,enero,2071,Alta,1061.580205,3724.530431,500988,301193,187990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137512,16305,San Nicolás,163,Punilla,16,Ñuble,16203,Coelemu,Valle del Itata,162,...,2023,12,diciembre,17,Alta,833.567298,833.567298,790647,139183,19389
3137513,16305,San Nicolás,163,Punilla,16,Ñuble,16207,Treguaco,Resto región Ñuble,162,...,2023,12,diciembre,44,Alta,833.567298,833.567298,790647,139183,10967
3137514,16305,San Nicolás,163,Punilla,16,Ñuble,16302,Coihueco,Resto región Ñuble,163,...,2023,12,diciembre,108,Alta,833.567298,833.567298,790647,159556,23654
3137515,16305,San Nicolás,163,Punilla,16,Ñuble,16303,Ñiquén,Resto región Ñuble,163,...,2023,12,diciembre,47,Alta,833.567298,833.567298,790647,159556,32388


In [7]:
# Revisar NaNs

nan_por_columna = df_total.isnull().sum()
print("NaN por columna:\n", nan_por_columna)

total_nan = df_total.isnull().sum().sum()
print("Total de NaN en el DataFrame:", total_nan)

NaN por columna:
 CUT Comuna Origen            0
Comuna Origen                0
CUT Provincia Origen         0
Provincia Origen             0
CUT Region Origen            0
Region Origen                0
CUT Comuna Destino           0
Comuna Destino               0
Destino Turistico            0
CUT Provincia Destino        0
Provincia Destino            0
CUT Region Destino           0
Region Destino               0
Anio                         0
CUT Mes                      0
Mes                          0
Viajes Ocasionales           0
Temporada                    0
PIB Region Origen            0
PIB Region Destino           0
Total Temporada Region       0
Total Temporada Provincia    0
Total Temporada Comuna       0
dtype: int64
Total de NaN en el DataFrame: 0


In [6]:
df_total.to_csv('../data/processed/df_listo_para_eda.csv', sep=';', index=False)